In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy as sci

# Download the list of S&P500 stocks
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_data = pd.read_html(sp500_url)[0]
sp500_symbols = sp500_data['Symbol'].tolist()

In [2]:
# Download daily historical stock prices for S&P500 stocks
# Replace 'YOUR_DATA_DIRECTORY' with the actual location of your data directory
data_directory = r'data'

for symbol in sp500_symbols:
    stock_data = pd.read_csv(f'{data_directory}.csv', index_col='Date', parse_dates=True)

    # Calculate daily returns
    stock_data['Return'] = stock_data['Close'].pct_change()

    # Calculate forward returns
    stock_data['Forward Return'] = stock_data['Close'].shift(-1) / stock_data['Close'] - 1

    # Calculate past week return
    stock_data['Past Week Return'] = stock_data['Return'].rolling(window=5).mean()

In [3]:
# Sort stocks into 10 bins based on prior week returns
bins = np.linspace(-0.1, 0.2, 11)
stock_data['Bin'] = pd.cut(stock_data['Past Week Return'], bins=bins, labels=range(10))

# Form daily-rebalanced, equally-weighted portfolios for each bin
portfolio_returns = {}

for bin_num in range(10):
    portfolio_data = stock_data[stock_data['Bin'] == bin_num]

    try:
        portfolio_data['Portfolio Weight'] = 1 / portfolio_data.shape[0]
    except ZeroDivisionError:
        portfolio_data['Portfolio Weight'] = 0

    # Use `agg()` to combine portfolio weight and mean return
    portfolio_return_series = portfolio_data.groupby('Date')['Portfolio Weight'].agg(['mean', lambda x: x * portfolio_data['Return'].mean()])
    portfolio_return_series.columns = ['Portfolio Weight', 'Portfolio Return']

    portfolio_returns[bin_num] = portfolio_return_series


C:\Users\a.omidzade\AppData\Local\Temp\ipykernel_9748\3512080053.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio_data['Portfolio Weight'] = 1 / portfolio_data.shape[0]
C:\Users\a.omidzade\AppData\Local\Temp\ipykernel_9748\3512080053.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio_data['Portfolio Weight'] = 1 / portfolio_data.shape[0]
C:\Users\a.omidzade\AppData\Local\Temp\ipykernel_9748\3512080053.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [4]:
portfolio_averages = pd.DataFrame(columns=['Bin', 'Average Return'])

for bin_num in range(10):
    portfolio_avg = portfolio_returns[bin_num]['Portfolio Return'].mean()
    portfolio_averages = portfolio_averages.append({'Bin': bin_num, 'Average Return': portfolio_avg}, ignore_index=True)

# Form a long-short portfolio of bins 1 and 10
long_short_portfolio = portfolio_returns[0]['Portfolio Return'] - portfolio_returns[9]['Portfolio Return']

# Test if the long-short portfolio has positive average returns
t_stat, p_value = sci.stats.ttest_1samp(long_short_portfolio, 0.05)

if p_value < 0.05:
    print("The long-short portfolio has a statistically significant positive average return.")
else:
    print("The long-short portfolio does not have a statistically significant positive average return.")

The long-short portfolio does not have a statistically significant positive average return.


C:\Users\a.omidzade\AppData\Local\Temp\ipykernel_9748\4202515839.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  portfolio_averages = portfolio_averages.append({'Bin': bin_num, 'Average Return': portfolio_avg}, ignore_index=True)
C:\Users\a.omidzade\AppData\Local\Temp\ipykernel_9748\4202515839.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  portfolio_averages = portfolio_averages.append({'Bin': bin_num, 'Average Return': portfolio_avg}, ignore_index=True)
C:\Users\a.omidzade\AppData\Local\Temp\ipykernel_9748\4202515839.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  portfolio_averages = portfolio_averages.append({'Bin': bin_num, 'Average Return': portfolio_avg}, ignore_index=True)
C:\Users\a.omidzade\AppData\Loc